author: @supercaizehua
- - -
# leetcode-347-top-k-frequent-elements
## 题意
输入 非空 整数数组      
输出 出现频率前k高的元素的列表
     
其它条件     
- 给定的k总是合理的,  1 ≤ k ≤ 数组中不相同的元素的个数
- 时间必须优于O(nlogn)
- 返回的元素列表顺序任意, 元素集合唯一即可

## solution 0 直接排序
时间O(nlogn)

In [ ]:

'''
直接排序
时间O(nlogn)
'''
class Solution0:
    def topKFrequent(self, nums, k):
        from collections import Counter
        count = Counter(nums)
        return [i[0] for i in count.most_common(k)]

## solution 1 heap
k = 1 时, hash表维护出现频率,返回最高    
k>1 需要根据出现频率快速获取元素的数据结构, 即优先队列                
1. 构建hashmap k-v:元素-频率                
2. 构建堆, 堆排序            


In [ ]:
class Solution1:
    def topKFrequent(self, nums, k):
        from collections import Counter
        import heapq
        count = Counter(nums)
        return heapq.nlargest(k, count.keys(), key= count.get)
        

In [ ]:
'''
手动维护大小为k的堆, 小根堆
时间O(nlogk)
'''
class Solution1_2:
    def topKFrequent(self, nums, k):
        from collections import Counter
        import heapq
        count = Counter(nums)
        heap = []
        for key, value in count.items():
            if len(heap) == k:
                if heap[0][0] < value:
                    heapq.heapreplace(heap, (value, key))
            else:
                    heapq.heappush(heap, (value, key))
        return [i[1] for i in heap]

In [ ]:
'''
手动维护大小为n-k的堆 大根堆
时间O(nlog(n-k))

从m个元素中, 通过堆选出最大的k个数
堆满后, 若新加的数小于堆顶, 堆顶加入结果, 否则新加的数加入结果
'''
class Solution1_3:
    def topKFrequent(self, nums, k):
        from collections import Counter
        import heapq
        count = Counter(nums)
        heap = []
        ans = []
        n = len(count)
        for num, freq in count.items():
            if len(heap) == n-k:
                if heap and freq < -heap[0][0]: #新加元素的freq 和堆顶元素比较
                    ans.append(heapq.heapreplace(heap, (-freq, num))[1])
                else:
                    ans.append(num)
            else:
                heapq.heappush(heap, (-freq, num))
        return ans

In [ ]:
class Solution1_4:
    def topKFrequent(self, nums, k):
        from collections import Counter
        import heapq
        count = Counter(nums)
        heap, ans = [],[]
        n = len(count)
        return self.bigheap(count, heap, ans, n, k) if k > n/2 else self.smallheap(count, heap, k)
    
    def bigheap(self, count, heap, ans, n, k):
        for num, freq in count.items():
            if len(heap) == n - k:
                if heap and freq < -heap[0][0]:
                    ans.append(heapq.heapreplace(heap, (-freq, num))[1])
                else:
                    ans.append(num)
            else:
                heapq.heappush(heap, (-freq, num))
        return ans

    def smallheap(self, count, heap, k):
        for num, freq in count.items():
            if len(heap) == k:
                if heap[0][0] < freq:
                    heapq.heapreplace(heap, (freq, num))
            else:
                    heapq.heappush(heap, (freq, num))
        return [i[1] for i in heap]

## S2: 快排思想, 快速选择
1. 获取 hashmap
2. 快速选择

时间O(n)


In [ ]:
class Solution2:
    def topKFrequent(self, nums, k):
        from collections import Counter
        count = Counter(nums)
        nums = [(freq, num) for num, freq in count.items()]

        kths = self.findKth(nums, k, 0, len(nums) - 1)
        return [i[1] for i in kths]

    def findKth(self, nums, k, low, high):
        if low >= high: return nums[:low+1]
        pivot = nums[low]
        i = low
        for j in range(low+1, high+1):
            if nums[j][0] > pivot[0]:
                nums[j], nums[i+1]  = nums[i+1], nums[j]
                i += 1
        nums[low], nums[i] = nums[i], nums[low]
        if i == k-1:
            return nums[:i+1]
        elif i > k-1:
            return self.findKth(nums, k, low, i-1)
        else:
            return self.findKth(nums, k, i+1, high)

In [ ]:
class Solution2_2:
    def topKFrequent(self, nums, k):
        from collections import Counter
        count = Counter(nums)
        nums = [(freq, num) for num, freq in count.items()]

        ans = self.quickselect(nums, k, 0, len(nums) - 1)
        return [i[1] for i in ans]

    def quickselect(self, nums, k, low, high):
        if k>= len(nums): return nums
        pivot_index = self.partition(nums, low, high)
        if pivot_index == k:
            return nums[:k]
        elif pivot_index > k:
            return self.quickselect(nums, k, low, pivot_index - 1)
        else:
            return self.quickselect(nums, k, pivot_index + 1, high)

    def partition(self, nums, low, high):
        pivot = nums[low]
        while low < high:
            while low < high and nums[high][0] <= pivot[0]:
                high -= 1
            nums[low] = nums[high]
            while low < high and nums[low][0] >= pivot[0]:
                low += 1
            nums[high] = nums[low]
        nums[high] = pivot
        return high
